# Prototype: Data Preparation

In [1]:
import open3d as o3d
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable

import sys
sys.path.insert(0, '../impl/utils/')
import voxel_processing as vp

batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# device will determine whether to run the training on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


device(type='cuda')

In [2]:
# Create datasets
vp.preprocess_files('../data/obj_test_data/', '../data/voxel_data')

Processing: ../data/obj_test_data/voxel_grid_1.obj -> ../data/voxel_data/voxel_grid_0
Processing: ../data/obj_test_data/voxel_grid_0.obj -> ../data/voxel_data/voxel_grid_1
2 file(s) have been processed.


In [25]:
# Load datasets
data = vp.read_dataset_from_path('../data/voxel_data/')
data = vp.prepare_dataset(data)
data

Reading: ../data/voxel_data/voxel_grid_1.obj
Reading: ../data/voxel_data/voxel_grid_0.omap
Reading: ../data/voxel_data/voxel_grid_1.offsetvec
Reading: ../data/voxel_data/voxel_grid_0.gridpoints
Reading: ../data/voxel_data/voxel_grid_0.obj
Reading: ../data/voxel_data/voxel_grid_1.omap
Reading: ../data/voxel_data/voxel_grid_0.offsetvec
Reading: ../data/voxel_data/voxel_grid_1.gridpoints
2 dataset(s) have been processed. 
2 dataset(s) have been prepared. 


[(TriangleMesh with 6130 points and 8448 triangles.,
  tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          ...,
  
          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
  

In [3]:
mesh, omap, grid_points, offset_vector, sample_points = data[0]

pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(np.array(sample_points[:] + offset_vector)))

o3d.visualization.draw_geometries([mesh, pcd])

In [59]:
a = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]], dtype=torch.float32)
a

tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.]],

        [[ 7.,  8.,  9.],
         [10., 11., 12.]]])

In [47]:
a[:, 0].shape

torch.Size([2, 3])

In [58]:
primitive_coords = torch.linspace(1, 13, 24)
primitive_coords

tensor([ 1.0000,  1.5217,  2.0435,  2.5652,  3.0870,  3.6087,  4.1304,  4.6522,
         5.1739,  5.6957,  6.2174,  6.7391,  7.2609,  7.7826,  8.3043,  8.8261,
         9.3478,  9.8696, 10.3913, 10.9130, 11.4348, 11.9565, 12.4783, 13.0000])

In [60]:
n_samples = a.shape[1]
tmp0 = a.transpose(1, 2)

xs = tmp0[:, 0].reshape((1, -1))
ys = tmp0[:, 1].reshape((1, -1))
zs = tmp0[:, 2].reshape((1, -1))

x = torch.searchsorted(primitive_coords, xs) - 1
y = torch.searchsorted(primitive_coords, ys) - 1
z = torch.searchsorted(primitive_coords, zs) - 1

x[x < 0] = 0
y[y < 0] = 0
z[z < 0] = 0

x = x.reshape((-1, 1, n_samples))
y = y.reshape((-1, 1, n_samples))
z = z.reshape((-1, 1, n_samples))

torch.concat([x, y, z], dim=1).transpose(1, 2)

tensor([[[ 0,  1,  3],
         [ 5,  7,  9]],

        [[11, 13, 15],
         [17, 19, 21]]])